In [ ]:
!pip install crewai crewai_tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from google.colab import userdata
OPENAI_MODEL_NAME = userdata.get('')
OPENAI_API_KEY = userdata.get('')


import os
import yaml
from crewai import Agent, Task, Crew

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': '/content/agents.yaml',
    'tasks': '/content/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [ ]:
from crewai_tools import SerperDevTool,ScrapeWebsiteTool, WebsiteSearchTool



In [ ]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
os.environ['OPENAI_API_KEY'] = ''
groq_llm = "groq/llama-3.3-70b-versatile"
os.environ['SERPER_API_KEY'] = ''
# os.environ["GROQ_API_KEY"] = ""  # Replace "YOUR_GROQ_API_KEY" with your actual Groq API key.

In [ ]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],

)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

In [ ]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Blockchain use and its impacts on economy.'
})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 61bdd370-e353-4db3-9108-90cf6c9e8aa5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 7be8708c-c9de-40aa-b458-92a925764ccd
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 7be8708c-c9de-40aa-b458-92a925764ccd
       Status: Executing Task...
    └── 🤖 Agent: Lead Market Analyst
        
            Status: In Progress

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Blockchain use and its impacts on economy.. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Blockchain use and its impacts on economy..



🚀 Crew: crew
└── 📋 Task: 7be8708c-c9de-40aa-b458-92a925764ccd
       Status: Executing Task...
    └── 🤖 Agent: Lead Market Analyst
        
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Chief Content Officer
## Final Answer: 
{
  "article": "# Comprehensive Analysis Report: The Impact of Blockchain on the Economy\n\n## 1. Executive Summary\nBlockchain technology is increasingly recognized as a transformative force in the global economy, with projections suggesting a potential increase of $2.1 trillion in global GDP by 2030. The report explores the latest data and trends related to blockchain technology, highlighting its implications for market movements, investor sentiment, and economic indicators. Its potential benefits could reshape financial systems and business processes across the globe, particularly in emerging markets.\n\n## 2. Key Market Trends\n- **Growth Projections**: The blockchain technology market is projected to grow from an estimated value of $2.5 billion in 2022 to approximately $77 billion by 2033. This explosive growth is driven by increased venture capital investments and the expanding application of blockchain solutions across various s

🚀 Crew: crew
├── 📋 Task: 7be8708c-c9de-40aa-b458-92a925764ccd
│      Assigned to: Lead Market Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Lead Market Analyst
│       
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (52)
│       └── 🔧 Used Read website content (6)
├── 📋 Task: 084aad6d-154f-4647-8ef8-e142f576cb6a
│      Assigned to: Chief Data Strategist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Chief Data Strategist
│       
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (54)
│       └── 🔧 Failed Search in a specific website (18)
├── 📋 Task: c107df01-92d2-4c52-90aa-f51db4d9b4a8
│      Assigned to: Creative Content Director
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Content Director
│       
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (55)
└── 📋 Task: b97d2130-4b7b-4d49-bef1-f7aeee0cdf77
       Status: Executing Task...
    └── 🤖 Agent: Chief Content Officer
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 7be8708c-c9de-40aa-b458-92a925764ccd
│      Assigned to: Lead Market Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Lead Market Analyst
│       
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (52)
│       └── 🔧 Used Read website content (6)
├── 📋 Task: 084aad6d-154f-4647-8ef8-e142f576cb6a
│      Assigned to: Chief Data Strategist
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Chief Data Strategist
│       
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (54)
│       └── 🔧 Failed Search in a specific website (18)
├── 📋 Task: c107df01-92d2-4c52-90aa-f51db4d9b4a8
│      Assigned to: Creative Content Director
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Content Director
│       
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (55)
└── 📋 Task: b97d2130-4b7b-4d49-bef1-f7aeee0cdf77
       Assigned to: Chief Content Officer
    
       Status: ✅ Completed
    └── 🤖 Agent: Chief Content Officer
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b97d2130-4b7b-4d49-bef1-f7aeee0cdf77                                                                     │
│  Agent: Chief Content Officer                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 61bdd370-e353-4db3-9108-90cf6c9e8aa5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
🛠️ Exploring how #Blockchain is set to transform
the economy! With a potential GDP boost of $2.1
trillion by 2030, the implications for businesses
are enormous. Check out our comprehensive
analysis! [link] #TechForGood
--------------------------------------------------
LinkedIn
The rise of #Blockchain technology could signify a
new era for economic growth. With emerging markets
leading the way and projected savings of $10
billion in transaction costs, the time to engage
with this technology is NOW. Read our in-depth
analysis to uncover key insights and strategies
for leveraging blockchain! [link]
--------------------------------------------------
Facebook
🌍 How will #Blockchain affect your business? Our
latest report delves into the transformative
impacts of this technology, projected to increase
global GDP by $2.1 trillion by 2030! Discover the
opportunities, risks, and what it means for the
future of finance and beyond. Read more here:
[link] #BlockchainRevolution
-----------

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Comprehensive Analysis Report: The Impact of Blockchain on the Economy

## 1. Executive Summary
Blockchain technology is increasingly recognized as a transformative force in the global economy, with projections suggesting a potential increase of $2.1 trillion in global GDP by 2030. The report explores the latest data and trends related to blockchain technology, highlighting its implications for market movements, investor sentiment, and economic indicators. Its potential benefits could reshape financial systems and business processes across the globe, particularly in emerging markets.

## 2. Key Market Trends
- **Growth Projections**: The blockchain technology market is projected to grow from an estimated value of $2.5 billion in 2022 to approximately $77 billion by 2033. This explosive growth is driven by increased venture capital investments and the expanding application of blockchain solutions across various sectors.

- **Regional Insights**: Emerging markets in Asia, South America, and Africa are expected to experience significant benefits from blockchain adoption. This trend signifies a shift towards new economic landscapes and opportunities as these countries leverage the technology for economic growth.

- **Financial Sector Transformation**: Blockchain technology could significantly reduce operational costs, particularly in cross-border transactions, with potential savings of up to $10 billion by streamlining payment processes. The World Economic Forum anticipates that as much as 10% of global GDP could be tokenized by 2027, marking a seismic shift in financial practices and infrastructures.

## 3. Opportunities and Risks
- **Opportunities**:
  - **Security and Transparency**: One of blockchain's standout features is its ability to enhance security and transparency, consequently reducing fraud risks and operational costs for businesses engaging in transactions.
  - **Innovation in Finance**: The increased adoption of Central Bank Digital Currencies (CBDCs) and decentralized finance (DeFi) solutions are set to revolutionize traditional financial systems, facilitating faster, cheaper, and more secure transactions for consumers.

- **Risks**:
  - **Regulatory Challenges**: Ongoing regulatory scrutiny remains a significant barrier to blockchain's wider adoption. The differing regulations across regions could stifle innovation and create uncertainty for businesses looking to implement blockchain solutions.
  - **Scalability Issues**: Building a decentralized network that maintains both security and scalability poses technical challenges that must be resolved for broader industry acceptance and use.

## 4. Actionable Insights
Based on the trends identified, the following actionable insights can be drawn:
- **Investment Strategies**: Investors should consider diversifying their portfolios to include blockchain-related assets, with a particular focus on startups and projects emerging from fast-growing markets.
- **Content Creation Focus**: Emphasize developing content that highlights both the benefits and risks of blockchain technology for businesses. Incorporating case studies and real-world applications will enhance relatability and understanding among varied audiences.
- **Educational Initiatives**: Companies should invest in educational initiatives aimed at informing stakeholders about the implications of blockchain technology, fostering a richer comprehension of its potential economic impacts.

## 5. Conclusion
As blockchain technology continues to evolve, its potential to reshape the global economy cannot be overstated. By synthesizing the recent market data and trends presented in this report, firms can position themselves strategically to exploit both the opportunities and challenges posed by this transformative technology.

## Data Visualizations
[Insert relevant charts/graphics that illustrate market growth, sector-specific impacts, and regulatory landscapes.]

## Recommendations for Content Creation
- Focus on practical applications of blockchain in various industries such as finance and supply chain.
- Highlight success stories from emerging markets that have adopted blockchain solutions successfully.
- Educate audiences regarding ongoing regulatory developments and their implications for businesses.